In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [2]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
#
n='Input_A6_024'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_A6_024
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0030139068589154225
test_rmse: 0.0030432632126253262
test_r2: 0.9407502866303064
feature number: 3
train_rmse: 0.002936537215829018
test_rmse: 0.0030670240095864625
test_r2: 0.9409285836880106
feature number: 4
train_rmse: 0.0024842079978108604
test_rmse: 0.0029344304259889486
test_r2: 0.9468793115236087
feature number: 5
train_rmse: 0.0024720743254105534
test_rmse: 0.002962965728289614
test_r2: 0.9461313716234795
feature number: 6
train_rmse: 0.0024642165567532624
test_rmse: 0.002965152907631811
test_r2: 0.9461414601234763
feature number: 7
train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199
feature number: 8
train_rmse: 0.002379564041432359
test_rmse: 0.003075669786845232
test_r2: 0.9418034900708916
feature number: 9
train_rmse: 0.0023770989483929087
test_rmse: 0.003085964749411889
test_r2: 0.9414905438092637
feature number: 10
train_rmse: 0.0023379359421082913
test_rmse: 0.0

train_rmse: 0.0018338224176942496
test_rmse: 0.003086749702970675
test_r2: 0.9410207490611029
feature number: 16
train_rmse: 0.0018318119933271502
test_rmse: 0.0030948674123207607
test_r2: 0.9408273660468384
feature number: 17
train_rmse: 0.0018298099829963678
test_rmse: 0.0030997690215639018
test_r2: 0.9407074629776744
feature number: 18
train_rmse: 0.001824894737825273
test_rmse: 0.003105028354166986
test_r2: 0.9405837783322403
feature number: 19
train_rmse: 0.0018204597411156163
test_rmse: 0.0031060738585166293
test_r2: 0.9404066913851414
feature number: 20
train_rmse: 0.0017909740413275422
test_rmse: 0.003254641784804776
test_r2: 0.933846078751279
Test_rmse_min = 1.fea_num: 7  2.rmse: 0.002902466068294067
Test_r2_max = 1. fea_num: 7  2.r2: 0.9479807856847199
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.0030139068589154225
test_rmse: 0.0030432632126253262
test_r2: 0.9407502866303064
feature number: 3
train_rmse: 0.002936537215829018
test_rmse: 0.0030670240095864625
test_r2

train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199
feature number: 8
train_rmse: 0.002379564041432359
test_rmse: 0.003075669786845232
test_r2: 0.9418034900708916
feature number: 9
train_rmse: 0.0023770989483929087
test_rmse: 0.003085964749411889
test_r2: 0.9414905438092637
feature number: 10
train_rmse: 0.0023379359421082913
test_rmse: 0.003377499975213054
test_r2: 0.929774997667978
feature number: 11
train_rmse: 0.0023357247216885093
test_rmse: 0.00338023609968926
test_r2: 0.9296412036069006
feature number: 12
train_rmse: 0.0023154945909019354
test_rmse: 0.003450549677376799
test_r2: 0.9261554887294026
feature number: 13
train_rmse: 0.0022450359444273044
test_rmse: 0.0036713822306884184
test_r2: 0.9108243595783018
feature number: 14
train_rmse: 0.0020311652715483005
test_rmse: 0.0032842106289582853
test_r2: 0.931954170179116
feature number: 15
train_rmse: 0.0018338224176942496
test_rmse: 0.003086749702970675
test_r2: 0.9410207490611029
featur

In [5]:
fea_number=7
#
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
#
y=data.Predict
skew_data=skew_pro(data)

num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_022', 'Input_A6_023', 'Input_A5_024', 'Input_A5_022',
       'Input_A5_023', 'Input_A4_024', 'Input_A4_022'], dtype=object)

## Linear Regression Baseline

In [6]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [7]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199


## Lasso

In [8]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [9]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_x,y,parameters)

[{'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}]


In [10]:
laso_rmse_ave(skew_x,y,9e-05)

test_rmse_ave: 0.002577104988079031
[0.0023685885869971367, 0.002794123595080413, 0.0036601488493290223, 0.0026809370729711153, 0.001895009090636438, 0.0024850153917334522, 0.0021799565004302097, 0.0015029419482945746, 0.002741303638521775, 0.003463025206796176]


test_r2_ave: 0.9592048946277887
[0.961111842859216, 0.9499281143239962, 0.9215162852268073, 0.9639303814141422, 0.9835564981566076, 0.9706223521009266, 0.9616757230341296, 0.9828025341054644, 0.9548802185651191, 0.942024996491478]


In [11]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 9e-05}, {'alpha': 9e-06}, {'alpha': 9e-06}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-06}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-06}, {'alpha': 9e-06}]


In [12]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.002858629886444398
[0.0022698036385402605, 0.0026546833454194245, 0.004410207615975337, 0.0025281182625454613, 0.002642067728980637, 0.0028450253357848038, 0.0021151745802722947, 0.002027588105765336, 0.002933624158560253, 0.004160006092600172]


test_r2_ave: 0.9499885212469665
[0.9642879587892206, 0.9548010681206917, 0.8860537267354931, 0.967925264694656, 0.9680361113595436, 0.9614937583590666, 0.9639196491529832, 0.9687003476063686, 0.9483272412105466, 0.9163400864410954]


## ElasticNet

In [13]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [17]:
alphas = [0.008]

#alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_x,y,alphas,l1ratio)

alpha
[0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008]


l1_ratio
[0.02, 0.03, 0.03, 0.02, 0.02, 0.03, 0.02, 0.02, 0.03, 0.02]


In [18]:
ElasticNet_rmse_ave(skew_x,y,[0.008],0.025)

test_rmse_ave: 0.0026544062838455527
[0.0026282055435210344, 0.002794101769195911, 0.0036094122365719307, 0.002741705403159541, 0.0020935989235555123, 0.0022614715964144653, 0.0023259347473728186, 0.0016319370984337889, 0.0030206462758751986, 0.003437049244355325]


test_r2_ave: 0.9567804501532479
[0.9521197121840798, 0.9499288965792457, 0.9236770710207742, 0.9622766819573948, 0.9799294770411244, 0.9756700548264, 0.956371189927127, 0.979723785585926, 0.9452161631817209, 0.9428914692286847]


In [19]:
alphas = [0.008]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_data[title],y,alphas,l1ratio)

alpha
[0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [20]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.002929450984140303
[0.002277065011807448, 0.003040880814779033, 0.004298547483106992, 0.0026067186890523296, 0.002628859786139357, 0.0029324636452928376, 0.0022889907356514637, 0.002207130616465935, 0.0030440251141509137, 0.0039698279449567235]


test_r2_ave: 0.947868570160925
[0.9640590991184206, 0.9406936059744937, 0.8917505962743009, 0.9658998224222778, 0.9683548934694092, 0.9590905040274521, 0.9577461404201756, 0.9629117693677869, 0.9443648626536585, 0.923814407881274]


## SVR

In [21]:
from sklearn.svm import SVR

In [22]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear', 'rbf','poly'], 'C':[1.2,1.3,1.1,1.4,1],'gamma': [1e-7,5e-6,5e-7],'epsilon':[0.2,0.1,0.05,0]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}




In [26]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear', 'rbf','poly'], 'C':[1.6,1.5,1.8,1.9,2],'gamma': [1e-3,1e-5,1e-4,1e-2],'epsilon':[0.1,0.05,0,0.2]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 0.0001, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 0.0001, 'kernel': 'rbf'}


best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 0.0001, 'kernel': 'rbf'}


best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 0.0001, 'kernel': 'rbf'}


best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}


best params
{'C': 2, 'epsilon': 0, 'gamma': 1e-05, 'kernel': 'rbf'}




In [23]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    y = data['Predict']
    if fea_num != None:
        num=data.select_dtypes(exclude='object')
        numcorr=num.corr()
        title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
        X= data[title]
    else:
        X= data.drop(['Predict'], axis=1)
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [24]:
model=SVR(kernel='linear', C=1.1, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.002494766253742296
test_rmse: 0.0029676558528763256
test_r2: 0.9455290284090244


In [27]:
model=SVR(kernel='rbf', C=1.75, gamma= 5e-05, epsilon= 0)
svr_rmse_ave(skew_data,None,model)

train_rmse: 0.0019453368648654657
test_rmse: 0.003085571228448463
test_r2: 0.9441034854310784


## XGB

In [28]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [46]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[04:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 70}


[04:02:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

In [31]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['gbtree'],
#     'n_estimators':[50,60,70],
     'n_estimators':range(50,221,10),
}

other_params = {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[17:45:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 120}


[17:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 210}


[17:46:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 100}


[17:46:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 210}


[17:47:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 100}


[17:47:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'boost

### 調 max_depth / min_child_weight

In [48]:
# {'booster': 'dart', 'n_estimators': 60}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:04:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 4}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

In [33]:
# {'booster': 'dart', 'n_estimators': 160}

parameters_for_testing = {
#    'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
     'max_depth': [3],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 140, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[18:04:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_child_weight': 2}


[18:04:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_child_weight': 1}


[18:04:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_child_weight': 2}


[18:04:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_child_weight': 1}


[18:04:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_child_weight': 1}


[18:04:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 3, 'min_

### 調 gamma

In [49]:
# {'max_depth': 2, 'min_child_weight': 2}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:05:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



In [34]:
# {'max_depth': 7, 'min_child_weight': 3}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 140, 'max_depth': 3, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[18:06:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[18:06:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



### 調 subsample / colsample_bytree

In [51]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.3}


[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.9}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

In [36]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.7],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 140, 'max_depth': 3, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[18:20:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.7, 'subsample': 0.9}


[18:20:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.7, 'subsample': 0.4}


[18:20:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.7, 'subsample': 0.7}


[18:20:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.7, 'subsample': 0.9}


[18:20:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.7, 'subsample': 0.9}


[18:20:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

### reg_alpha / reg_lambda

In [52]:
# {'colsample_bytree': 0.5, 'subsample': 0.6}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:10:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:27]

In [37]:
# {'colsample_bytree': 0.9, 'subsample': 0.7}

parameters_for_testing = {
     'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#    'reg_alpha': [0.05],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 140, 'max_depth': 3, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.8,'colsample_bytree': 0.7}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[18:24:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[18:24:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[18:25:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 2}


[18:25:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[18:26:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 2}


[18:26:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[18:27:23] WAR

### learning rate

In [53]:
# {'reg_alpha': 0, 'reg_lambda': 0.7}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5,'reg_alpha': 0, 'reg_lambda': 0.7}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [38]:
# {'reg_alpha': 0.0, 'reg_lambda': 0.3}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'gbtree', 'n_estimators': 140, 'max_depth': 3, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.8,'colsample_bytree': 0.7,'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[18:31:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[18:31:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[18:31:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[18:31:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[18:31:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[18:31:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[18:31:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depr

In [131]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 60, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.85,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008496031540705907
[0.007648650629856374, 0.007861428699795805, 0.008923526006532567, 0.008422016710058338, 0.007514787893473913, 0.007671313900216453, 0.009908224384169528, 0.0094945394022681, 0.008777706642056635, 0.008738121138631347]


test_r2_ave: 0.5658204873179591
[0.6994093043442213, 0.6676009790074625, 0.49424336268772506, 0.5730727307503245, 0.6729951732468282, 0.6589820348516682, 0.3470225948052349, 0.46909149956225205, 0.5156770986673376, 0.5601100952565365]


In [39]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'gbtree', 
              n_estimators = 140, 
              max_depth = 3, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_x, y,best_xgb_model)


test_rmse_ave: 0.003640189428763986
[0.003339300877782445, 0.004516094158925662, 0.004664159620916192, 0.0033304265649229474, 0.004221610178794611, 0.0034449042129944566, 0.0025742767826867747, 0.0034859299145839216, 0.0028850073958914453, 0.003940184580141409]


test_r2_ave: 0.9199740419238209
[0.9227053915312531, 0.8691936889840521, 0.872553220528471, 0.9443368174812701, 0.918392894815119, 0.9435436032234329, 0.9465572503719341, 0.9074838935518704, 0.9500257179214473, 0.9249479408293585]


## RandomForest

In [40]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [78]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [79]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [81]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [20,40,50,60,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0688s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  64 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 108 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 134 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 190 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 220 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 254 tasks      | elapsed:    9.5s
[Parallel(n_jobs=6)]: Done 288 tasks      | elapse

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50}
best score
-6.733198261475197e-05


[Parallel(n_jobs=6)]: Done 1800 out of 1800 | elapsed:  1.4min finished


In [82]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    7.6s
[Parallel(

best params
{'max_leaf_nodes': 14, 'min_samples_leaf': 2}
best score
-6.728825706399575e-05


[Parallel(n_jobs=6)]: Done 585 out of 585 | elapsed:   24.0s finished


In [83]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 14,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.008359151931886303
[0.008046421179911343, 0.007522207654460264, 0.008970754200764595, 0.008192033878800358, 0.007261528804827015, 0.007389966886686526, 0.009434354784131698, 0.009095030814723662, 0.00923126977742949, 0.008447951337128053]


test_r2_ave: 0.5800138319443187
[0.6673317079609893, 0.6956681454215288, 0.4888757128331376, 0.5960708670050614, 0.6946648253743058, 0.6835371494626516, 0.40798747419915105, 0.5128303480153389, 0.46433192124209743, 0.5888401679289246]


In [41]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_x, y, test_size=0.2, random_state=42)

In [43]:
param_grid = {
              'n_estimators': [60,70,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    4.4s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    5.3s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    7.6s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    9.1s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   10.5s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   12.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   14.4s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   16.3s
[Parallel(

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 70}
best score
-1.5071849052441432e-05


In [44]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    2.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.7s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    3.5s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.4s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    5.7s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    7.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    8.7s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   10.2s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   12.4s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   14.5s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   16.5s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   19.1s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   22.4s
[Parallel(

best params
{'max_leaf_nodes': 8, 'min_samples_leaf': 4}
best score
-1.4716714522177978e-05


In [45]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 8,
            min_samples_leaf =4,
            random_state = 42) 
rand_ave(skew_x, y,rand_model)

test_rmse_ave: 0.0034680579805540074
[0.002700952469770107, 0.0037247805773637705, 0.004797185118881184, 0.002730691949146187, 0.003764582049831371, 0.0034789958582865472, 0.0028115804270497785, 0.0031344888813612244, 0.002985785466879046, 0.0045515370069708575]


test_r2_ave: 0.9273508196906557
[0.9494324419360723, 0.9110176011615428, 0.8651797862056505, 0.9625791430217678, 0.9351058945439009, 0.9424206604935172, 0.9362501250179885, 0.9251979479767745, 0.9464733699476682, 0.8998512266016736]
